In [ ]:
import pandas as pd

df1 = pd.read_csv("/content/drive/MyDrive/Suicide_Detection.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Suicide_Ideation_Dataset(Twitter-based).csv")

In [ ]:
df1.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
df2.head()

,Tweet,Suicide
0,making some lunch,Not Suicide post
1,@Alexia You want his money.,Not Suicide post
2,@dizzyhrvy that crap took me forever to put to...,Potential Suicide post
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,Not Suicide post
4,Trying out &quot;Delicious Library 2&quot; wit...,Not Suicide post


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  232074 non-null  int64 
 1   text        232074 non-null  object
 2   class       232074 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.3+ MB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    1785 non-null   object
 1   Suicide  1787 non-null   object
dtypes: object(2)
memory usage: 28.1+ KB


In [ ]:
df1["class"].value_counts()

,count
class,
suicide,116037
non-suicide,116037


In [ ]:
df2["Suicide"].value_counts()

,count
Suicide,
Not Suicide post,1127
Potential Suicide post,660


In [ ]:
label_mapping = {
    "Not Suicide post": "non-suicide",
    "Potential Suicide post": "suicide"
}

In [ ]:
df2["Suicide"] = df2["Suicide"].map(label_mapping)

In [ ]:
df1 = df1.rename(columns={"text": "text", "class": "label"})
df2 = df2.rename(columns={"Tweet": "text", "Suicide": "label"})

In [ ]:
print(df1.isnull().sum())

Unnamed: 0    0
text          0
label         0
dtype: int64


In [ ]:
print(df2.isnull().sum())

text       2
label    660
dtype: int64


In [ ]:
df2 = df2.dropna(subset=["text", "label"])

In [ ]:
print(df2.isnull().sum())

text     0
label    0
dtype: int64


In [ ]:
merged_df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233200 entries, 0 to 233199
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  232074 non-null  float64
 1   text        233200 non-null  object 
 2   label       233200 non-null  object 
dtypes: float64(1), object(2)
memory usage: 5.3+ MB


In [ ]:
merged_df.head()

,Unnamed: 0,text,label
0,2.0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3.0,Am I weird I don't get affected by compliments...,non-suicide
2,4.0,Finally 2020 is almost over... So I can never ...,non-suicide
3,8.0,i need helpjust help me im crying so hard,suicide
4,9.0,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
merged_df["label"].value_counts()

,count
label,
non-suicide,117163
suicide,116037


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
        text = re.sub(r"\W", " ", text)
        text = re.sub(r"\s+", " ", text).strip()
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return " ".join(tokens)
    return ""

In [ ]:
merged_df["processed_text"] = merged_df["text"].apply(preprocess_text)

In [ ]:
merged_df[["text", "processed_text"]].head()

,text,processed_text
0,Ex Wife Threatening SuicideRecently I left my ...,ex wife threatening suiciderecently left wife ...
1,Am I weird I don't get affected by compliments...,weird get affected compliment coming someone k...
2,Finally 2020 is almost over... So I can never ...,finally 2020 almost never hear 2020 bad year e...
3,i need helpjust help me im crying so hard,need helpjust help im cry hard
4,"I’m so lostHello, my name is Adam (16) and I’v...",losthello name adam 16 struggling year afraid ...


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
max_words = 10000
max_len = 200

In [ ]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(merged_df["processed_text"])

In [ ]:
X = tokenizer.texts_to_sequences(merged_df["processed_text"])
X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post')

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

In [ ]:
y = label_encoder.fit_transform(merged_df["label"])
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
embedding_dim = 128

In [ ]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)